## IMPORTant Stuff

In [ ]:
import geopandas as gp
import folium

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import glob
import functools

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

In [4]:
pd.set_option('display.max_columns', None)


In [2]:
kick_data = pd.read_csv('../../data/02_intermediate/kick_inter.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11,15,16,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
kick_data.columns

Index(['backers_count', 'blurb', 'converted_pledged_amount', 'country',
       'created_at', 'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'last_update_published_at', 'launched_at', 'name', 'permissions',
       'pledged', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'unread_messages_count',
       'unseen_activity_count', 'urls', 'usd_pledged', 'usd_type',
       'sub_category', 'overall_category', 'city', 'country_loc', 'state_loc',
       'creator_name', 'creator_slug'],
      dtype='object')

In [25]:
kick_data.state.value_counts()

successful    139265
failed        121385
live           52687
canceled       15657
suspended       1018
Name: state, dtype: int64

In [27]:
kick_data.loc[((kick_data.state == 'successful') & (kick_data.state == 'failed'))]

,backers_count,blurb,converted_pledged_amount,country,created_at,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,last_update_published_at,launched_at,name,permissions,pledged,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,unread_messages_count,unseen_activity_count,urls,usd_pledged,usd_type,sub_category,overall_category,city,country_loc,state_loc,creator_name,creator_slug


In [9]:
kick_data.loc[(kick_data.country_loc=='US')].city.nunique()

6919

In [14]:
kick_data.city.value_counts()

Los Angeles              9072
New York                 7657
London                   6914
Chicago                  3867
San Francisco            3292
Brooklyn                 3099
Portland                 2716
Seattle                  2635
Atlanta                  2155
Toronto                  2090
Boston                   2001
Austin                   1990
San Diego                1662
Philadelphia             1661
Minneapolis              1461
Denver                   1429
Washington               1422
Houston                  1384
Nashville                1380
Dallas                   1377
Vancouver                1329
Hong Kong                1234
Melbourne                1221
Paris                    1164
Salt Lake City           1110
Orlando                  1101
Manhattan                1078
Las Vegas                1062
Sydney                   1042
Phoenix                  1022
                         ... 
Denekamp                    1
Chimaltenango               1
Lauderdale

There are 6919 unique US cities represented in our dataframe. Using https://simplemaps.com/data/us-cities, I will aim to get the Long/Lat info for each city and add to this df. 

In [10]:
cities = pd.read_csv('../../data/01_raw/uscitiesv1.5.csv')

In [15]:
cities.loc[(cities.city == 'Los Angeles')]

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
8446,Los Angeles,Los Angeles,TX,Texas,48283,La Salle,28.4658,-99.0000,NaN,NaN,3.5,point,False,America/Chicago,78021,1840036276
16938,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12815475.0,3999759.0,3295.0,polygon,True,America/Los_Angeles,90291 90293 90292 91316 91311 90034 90037 9000...,1840020491


Thinking about the mask I would need to place on my dataframe. 

I need just US + unique City/State combinations. The latter 

In [16]:
kick_data.country_loc.isnull().sum() ## 1102 rows do not have a country defined - going to drop those.

1102

In [21]:
kick_data.dropna(subset = ['country_loc'], inplace = True)

In [22]:
kick_data.country_loc.isnull().sum()

0

In [24]:
kick_data.state_loc.isnull().sum()

93

In [26]:
kick_data.city.isnull().sum()

133447